In [1]:
# In this scripts, you will laern how to do calibartion and zero-shot learning
# We use manual verbalizer and knowledgeable verbalizer as examples.
from tqdm import tqdm
from openprompt.data_utils.text_classification_dataset import AgnewsProcessor
import torch
from openprompt.data_utils.utils import InputExample
from openprompt.trainer import ClassificationRunner

dataset = {}
dataset['train'] = AgnewsProcessor().get_train_examples("../datasets/TextClassification/agnews")
dataset['test'] = AgnewsProcessor().get_test_examples("../datasets/TextClassification/agnews")
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("roberta", "roberta-large")

In [2]:
# plm

In [3]:
# !pip install yacs
# !pip install tensorboardX
# !pip install rouge==1.0.0

In [4]:
dataset['train']

[{
   "guid": "0",
   "label": 2,
   "meta": {},
   "text_a": "Wall St. Bears Claw Back Into the Black (Reuters)",
   "text_b": "Reuters - Short-sellers, Wall Street's dwindling band of ultra-cynics, are seeing green again.",
   "tgt_text": null
 },
 {
   "guid": "1",
   "label": 2,
   "meta": {},
   "text_a": "Carlyle Looks Toward Commercial Aerospace (Reuters)",
   "text_b": "Reuters - Private investment firm Carlyle Group, which has a reputation for making well-timed and occasionally controversial plays in the defense industry, has quietly placed its bets on another part of the market.",
   "tgt_text": null
 },
 {
   "guid": "2",
   "label": 2,
   "meta": {},
   "text_a": "Oil and Economy Cloud Stocks' Outlook (Reuters)",
   "text_b": "Reuters - Soaring crude prices plus worries about the economy and the outlook for earnings are expected to hang over the stock market next week during the depth of the summer doldrums.",
   "tgt_text": null
 },
 {
   "guid": "3",
   "label": 2,
   "me

In [5]:

from openprompt.prompts import ManualTemplate
mytemplate = ManualTemplate(tokenizer=tokenizer).from_file("../scripts/TextClassification/agnews/manual_template.txt", choice=0)

from openprompt import PromptDataLoader



# ## Define the verbalizer
from openprompt.prompts import ManualVerbalizer, KnowledgeableVerbalizer

# this is a user provided set of label words?
myverbalizer = KnowledgeableVerbalizer(tokenizer, num_classes=4).from_file("../scripts/TextClassification/agnews/knowledgeable_verbalizer.txt")

In [6]:
len(myverbalizer.label_words)

4

In [7]:
mytemplate.text

[{'add_prefix_space': ' ', 'text': 'A'},
 {'add_prefix_space': ' ', 'mask': None},
 {'add_prefix_space': ' ', 'text': 'news :'},
 {'add_prefix_space': ' ', 'placeholder': 'text_a'},
 {'add_prefix_space': ' ', 'placeholder': 'text_b'}]

In [8]:
# load promptforclassification model class

from openprompt import PromptForClassification
promptModel = PromptForClassification(
    template = mytemplate,
    plm = plm,
    verbalizer = myverbalizer,
)

In [9]:
promptModel

PromptForClassification(
  (prompt_model): PromptModel(
    (plm): RobertaForMaskedLM(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0): RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): Linear(in_features=1024, out_features=1024, bias=True)
                  (key): Linear(in_features=1024, out_features=1024, bias=True)
                  (value): Linear(in_features=1024, out_features=1024, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
   

In [10]:
use_cuda = True
if use_cuda:
    promptModel = promptModel.cuda()

In [11]:
# prompt data loader

from openprompt import PromptDataLoader
train_dataloader = PromptDataLoader(
    dataset = dataset['train'],
    tokenizer = tokenizer, 
    template = mytemplate, 
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=512, decoder_max_length=3
)

test_dataloader = PromptDataLoader(
    dataset = dataset['test'],
    tokenizer = tokenizer, 
    template = mytemplate, 
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=512, decoder_max_length=3
)

tokenizing: 120000it [01:49, 1099.42it/s]
tokenizing: 7600it [00:06, 1142.69it/s]


In [12]:
# learning rate and optimizer
# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in promptModel.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in promptModel.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

In [13]:
def fit(model, train_dataloader, val_dataloader, loss_func, optimizer, epochs = 5, cuda = True):
    best_score = 0.0
    for epoch in range(epochs):
        train_epoch(model, train_dataloader, loss_func, optimizer, epoch, cuda = True)
        score = evaluate(model, val_dataloader)
        if score > best_score:
            best_score = score
    return best_score
        

def train_epoch(model, train_dataloader, loss_func, optimizer, epoch, cuda = True):
    model.train()
    tot_loss = 0
    pbar = tqdm(test_dataloader)
    for step, inputs in enumerate(pbar):
        if cuda:            
            inputs = inputs.cuda()
        logits = model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        tot_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if step %500 ==1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)

def evaluate(model, val_dataloader, cuda = True):
    model.eval()
    allpreds = []
    alllabels = []
    with torch.no_grad():
        for step, inputs in enumerate(val_dataloader):
            if cuda:
                
                inputs = inputs.cuda()
            logits = model(inputs)
            labels = inputs['label']
            alllabels.extend(labels.cpu().tolist())
            allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
    acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
    print(f"accuracy was: {acc}")
    return acc

In [14]:
# # run training
# for epoch in range(1):
#     tot_loss = 0 
#     for step, inputs in enumerate(train_dataloader):
#         if use_cuda:
#             inputs = inputs.cuda()
#         logits = promptModel(inputs)
#         labels = inputs['label']
#         loss = loss_func(logits, labels)
#         loss.backward()
#         tot_loss += loss.item()
#         optimizer.step()
#         optimizer.zero_grad()
#         if step %100 ==1:
#             print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)

In [ ]:
score = fit(promptModel, train_dataloader, test_dataloader, loss_func, optimizer, cuda = True)


test_score = evaluate(promptModel, test_dataloader)
print(f"Test score was: {test_score}")

  0%|          | 1/7600 [00:00<52:08,  2.43it/s]

Epoch 0, average loss: 2.115755945444107


  7%|▋         | 501/7600 [02:24<34:23,  3.44it/s]

Epoch 0, average loss: 1.4125970934145304


 13%|█▎        | 1001/7600 [04:49<31:57,  3.44it/s]

...

 53%|█████▎    | 4001/7600 [19:20<17:23,  3.45it/s]

Epoch 0, average loss: 1.4057508843131605


 59%|█████▉    | 4501/7600 [21:46<15:00,  3.44it/s]

Epoch 0, average loss: 1.4054806423990667


 63%|██████▎   | 4803/7600 [23:13<13:34,  3.44it/s]